In [14]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from os.path import join as oj
import analyze_utils
import matplotlib.pyplot as plt
import iprompt
from tqdm import tqdm
from nltk.translate.bleu_score import sentence_bleu
import pickle as pkl
tqdm.pandas()
keys = ['model_cls', 'task_name', 'checkpoint', 'seed', 'gt_prompt']
keys_extra = ['n_shots', 'num_learned_tokens']
keys_out = ['reciprocal_rank', 'prefixes', 'iprompt_preprefix_str']

# save_dir = '/home/chansingh/mntv1/iprompt_revision_xmas/'
save_dir = '/home/chansingh/mntv1/iprompt_revision_reranked/'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
# r = analyze_utils.load_results_and_cache_autoprompt_json(save_dir, save_file='r.pkl', one_row_only=True)
r = pd.read_pickle(oj(save_dir, 'r.pkl'))

In [17]:
# # subselect the data
r = r[r['n_shots'] == 5]
r = r[r['num_learned_tokens'] == 6]
# original suffix had a reranking step, correct one is just called "suff"
r = r[~(r['model_cls'] == 'suffix')]

# # add more keys
keys_derived = ['task_collection', 'top_prompt_correctness', 'bleu_top_prompt']
r['task_collection'] = r.task_name.apply(analyze_utils.task_collection)
r['top_prompt_correctness'] = r['reciprocal_rank'] == 1
r['gt_prompt'] = r['task_name'].apply(
    lambda x: iprompt.data.TASKS[x]['description'])
r['bleu_top_prompt'] = r.progress_apply(
    lambda x: sentence_bleu([x['gt_prompt'].split()],
                            x['prefixes'].split(), weights=(0.75, 0.25)), axis=1)  # maybe need to split before calling?

d = r[keys + keys_extra + keys_out + keys_derived]
d.to_pickle(oj(save_dir, 'd.pkl'))
d = pd.read_pickle(oj(save_dir, 'd.pkl'))


  0%|          | 0/1577 [00:00<?, ?it/s]/home/chansingh/.embgam/lib/python3.8/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
100%|██████████| 1577/1577 [00:00<00:00, 22047.91it/s]


# Metrics table

In [18]:
metrics = d.groupby(by=['model_cls', 'task_collection', 'checkpoint']).mean(numeric_only=True).reset_index()
ms = []
# with pd.option_context('display.float_format', lambda x: '%.1e' % x):
with pd.option_context('display.float_format', lambda x: '%.2g' % x):
    for ckpt in metrics.checkpoint.unique():
        m = (
            metrics[metrics.checkpoint == ckpt]
            .pivot_table(index='model_cls', columns='task_collection', values=['reciprocal_rank', 'top_prompt_correctness', 'bleu_top_prompt'])
            .transpose()
        )
        display(m)
        ms.append(m)

model_cls                               autoprompt  iprompt    suff
                       task_collection                             
bleu_top_prompt        ANLI                7.6e-80    0.008 0.00071
                       DD                  3.1e-80  6.4e-79   0.002
                       Induction           1.6e-79    0.028 0.00096
                       Math                      0  2.1e-79  0.0078
reciprocal_rank        ANLI                  0.039     0.33    0.06
                       DD                   0.0036    0.016   0.018
                       Induction             0.096     0.31   0.035
                       Math                   0.17     0.63   0.075
top_prompt_correctness ANLI                  0.037     0.31   0.011
                       DD                        0        0  0.0056
                       Induction             0.096     0.27  0.0089
                       Math                   0.17     0.53   0.034

model_cls                               autoprompt  iprompt    suff
                       task_collection                             
bleu_top_prompt        ANLI                1.8e-79   0.0047  0.0013
                       DD                        0  5.3e-79 3.8e-79
                       Induction                 0    0.031 0.00052
                       Math                      0  1.3e-78 1.2e-78
reciprocal_rank        ANLI                  1e-10      0.3   0.071
                       DD                   0.0002    0.062   0.058
                       Induction               0.4     0.34   0.028
                       Math                   0.33     0.13   0.028
top_prompt_correctness ANLI                      0     0.29       0
                       DD                        0    0.053   0.043
                       Induction               0.4     0.33       0
                       Math                   0.33        0       0

model_cls                               iprompt    suff
                       task_collection                 
bleu_top_prompt        ANLI              0.0099 7.4e-80
                       DD                     0 2.3e-79
                       Induction          0.034 0.00028
                       Math               0.065   1e-78
reciprocal_rank        ANLI                0.35   0.066
                       DD                  0.12   0.008
                       Induction           0.25   0.079
                       Math                0.15   0.033
top_prompt_correctness ANLI                0.33   0.045
                       DD                     0       0
                       Induction           0.25   0.032
                       Math                 0.1       0

In [19]:
for m in ms:
    # put iprompt column first
    col = m['iprompt']
    m = m.drop(columns='iprompt')
    m.insert(loc=0, column='iprompt', value=col)

    # printing index
    mets_ordered = ['reciprocal_rank',
                    'top_prompt_correctness', 'bleu_top_prompt']
    collections_ordered = ['Math', 'ANLI', 'Induction']
    index = pd.MultiIndex.from_product([mets_ordered, collections_ordered])
    m[m < 1e-4] = 0
    m_s = (
        m
        .loc[index]  # reorder
        .applymap(lambda x: str.format("{:0.2g}", x))
        .apply(lambda x:
               [f"\\textbf{{{i}}}" if x.name ==
                'iprompt' else i for i in x],
               axis=0)
        .style.to_latex(hrules=True)
        .replace(' & task_collection &  &  &  \\\\\n', '')
        .replace(' & model_cls & iprompt & autoprompt & suff',  ' & & iPrompt & AutoPrompt & Suffix')
    )
    rename = {
        'reciprocal_rank': 'MRR',
        'top_prompt_correctness': 'Correctness',
        'bleu_top_prompt': 'BLEU',
    }
    for k in rename.keys():
        s = '\multirow[c]{3}{*}{' + k + '}'
        # print(s)
        m_s = m_s.replace(
            '\multirow[c]{3}{*}{' + k + '}',
            '\parbox[c]{0.3mm}{\multirow{3}{*}{\\rotatebox[origin = c]{90} {' + rename[k] + '}}}'
        )

    print(m_s)


\begin{tabular}{lllll}
\toprule
 & & iPrompt & AutoPrompt & Suffix \\
\midrule
\parbox[c]{0.3mm}{\multirow{3}{*}{\rotatebox[origin = c]{90} {MRR}}} & Math & \textbf{0.63} & 0.17 & 0.075 \\
 & ANLI & \textbf{0.33} & 0.039 & 0.06 \\
 & Induction & \textbf{0.31} & 0.096 & 0.035 \\
\parbox[c]{0.3mm}{\multirow{3}{*}{\rotatebox[origin = c]{90} {Correctness}}} & Math & \textbf{0.53} & 0.17 & 0.034 \\
 & ANLI & \textbf{0.31} & 0.037 & 0.011 \\
 & Induction & \textbf{0.27} & 0.096 & 0.0089 \\
\parbox[c]{0.3mm}{\multirow{3}{*}{\rotatebox[origin = c]{90} {BLEU}}} & Math & \textbf{0} & 0 & 0.0078 \\
 & ANLI & \textbf{0.008} & 0 & 0.00071 \\
 & Induction & \textbf{0.028} & 0 & 0.00096 \\
\bottomrule
\end{tabular}

\begin{tabular}{lllll}
\toprule
 & & iPrompt & AutoPrompt & Suffix \\
\midrule
\parbox[c]{0.3mm}{\multirow{3}{*}{\rotatebox[origin = c]{90} {MRR}}} & Math & \textbf{0.13} & 0.33 & 0.028 \\
 & ANLI & \textbf{0.3} & 0 & 0.071 \\
 & Induction & \textbf{0.34} & 0.4 & 0.028 \\
\parbox[c]{0.3mm

# Prompts table

In [20]:
for ckpt in d.checkpoint.unique():
    print(ckpt)
    prompts = d[d.checkpoint == ckpt]
    print(prompts.shape[0])
    prompts = (
        prompts
        .sort_values(by=['task_collection', 'reciprocal_rank'], ascending=False)
        .groupby(by=['model_cls', 'task_name'])
        .first()
    )
    # with pd.option_context('display.float_format', lambda x: '%.1e' % x):
    with pd.option_context('display.max_rows', None):
        display(prompts[['prefixes']].pivot_table(index='task_name', columns='model_cls', values='prefixes', aggfunc=lambda x: ' '.join(x)))

EleutherAI/gpt-j-6B
1144


model_cls,autoprompt,iprompt,suff
task_name,,,
active_to_passive,(= ■...]the the subst,Complete sentences using nouns to,Create a sentence or group of
add_two,fmReturns Adding together<|endoftext|> accomplish,Add two values together to get,n>2 m1
antonyms,shothe becthe But But,The noun to its opposite (,The code to ascend. You
cause_and_effect,Them Kinect commercially developed mosquit The,Choose an organization to which you,write programs that read through an
common_concept,????????the configuredcffff Should????????,which of the following are words,which is a common word used
d3_0,NaN,text contains answer=no (,string was in one of following
d3_1,NaN,"would always print ""no.""",string doesn't match any template
d3_10,Peter yelled nonexistent�_-_ Cand,question contains 'no _______,contains all of these words or
d3_11,caused senatorSpoiler moderators Bounty inter...,"phrase "" i"" does not",was matched; output otherwise No


google/flan-t5-xl
197


model_cls,iprompt,suff
task_name,,
active_to_passive,Give the most detailed description of,Answer: You cannot do this
add_two,Prompt: 18 years,NaN
antonyms,NaN,Answer: capitalism. Capitalis
cause_and_effect,The man sipped the,Answer: You cannot do
d3_0,NaN,Answer: yes publique à
d3_1,NaN,Answer: yes publique station/
d3_11,NaN,no.|no
d3_12,NaN,yes|;e finally and
d3_13,NaN,It's impossible to say


facebook/galactica-6.7b
236


model_cls,autoprompt,iprompt,suff
task_name,,,
add_two,impl�<s>PageIndexfre summation,NaN,NaN
antonyms,NaN,What is the opposite of?,Is the answer to this activity
cause_and_effect,infestation deli difthe Diff capped,NaN,Is that true or false?
common_concept,NaN,A list of words related by,[...] you need
d3_0,footnotesize oedema attributeCorollary dise veto,NaN,contain sarcastic content otherwise mark
d3_1,Origin Obl Engel Reachfeaturethe,NaN,says something like these characters love
d3_10,AN relation BCEdi transition forbidden,NaN,sentence includes all possible answers with
d3_11,GLS incomp Rend advisory Regulates head,NaN,contain answer y n otherwise Question
d3_12,assert uncoETX antich Designs counterex,NaN,sentence includes all three keywords with
